In [ ]:
import nltk
nltk.download()
# here to download movie_reviews and stopwords packages

In [ ]:
print('step 1: ')
from nltk.corpus import movie_reviews as mr
#print(mr.readme())
# divide the movie_reviews into 2 categories: positive and negative
positive = mr.fileids('pos')
negative = mr.fileids('neg')
# form a new movie_reviews as a dictionary by positive and negative
new_mr = dict(pos = positive, neg = negative)
# the length of positive and negative lists are both 1000
posLen = len(new_mr['pos'])
negLen = len(new_mr['neg'])
print('movie reviews are loaded with ')
print('%s positive reviews ' % posLen)
print('%s negative reviews ' % negLen)
print()

step 1: 
movie reviews are loaded with 
1000 positive reviews 
1000 negative reviews 



In [ ]:
print('step 2: ')
from nltk.corpus import stopwords
sw = stopwords.words('english');
# here in the stopwords list 'sw'
# it should remove some words from the list
# such as: 'no','nor','not','too', and
# "don't","aren't", "couldn't", 'didn', "didn't", 
# "doesn't", "hadn't", "hasn't", "haven't", "isn't", "mightn't",
#  "mustn't", "needn't", "shan't", "shouldn't", "wasn't", "weren't",
#  "won't", "wouldn't"
sw.remove('no')
sw.remove('nor')
sw.remove('not')
sw.remove('too')
sw1 = []
for v in sw:
    if not v.endswith("n't"):
        sw1.append(v)
    #else:
        #print(v)
sw1.remove('ain')
sw1.remove('aren')
sw1.remove('couldn')
sw1.remove('didn')
sw1.remove('doesn')
sw1.remove('hadn')
sw1.remove('haven')
sw1.remove('isn')
sw1.remove('mightn')
sw1.remove('hasn')
sw1.remove('mustn')
sw1.remove('needn')
sw1.remove('shan')
sw1.remove('wasn')
sw1.remove('weren')
sw1.remove('wouldn')
sw = sw1
#print(len(sw))
print('there are %s stopwords loaded' % len(sw1))

# input a word, 
# determine whether it is a stopword or not
# remove all punctuations from the word
def pre_process(word):
  # remove punctuations except apostrophe
  w = ""
  hasAlphabet = False
  for i in word:
    if i >= 'a' and i <= 'z':
      w = w + i
      hasAlphabet = True
    elif i >= 'A' and i <= 'Z':
      w = w + i
      hasAlphabet = True
    elif i == '\'': # apostrophe
      w = w + i
  #print(w)
  # check whether it is stopword
  if hasAlphabet and w not in sw:
    return w.lower()
  return ''

# selection the 3000 most important words.
# find the 3000 most important words by frequency
# top, is how many words in the list to be returned
def importantWords(corpus, file, top):
  iw = []
  words_dic = {}
  pLen = len(file['pos'])
  i = 0
  while i < pLen:
    movie_review = corpus.words(file['pos'][i])
    for w in movie_review:
      w = pre_process(w)
      if w not in words_dic:
        words_dic[w] = 1
      else:
        words_dic[w] = words_dic[w] + 1
    i = i + 1
  nLen = len(file['neg'])
  i = 0
  while i < nLen:
    movie_review = corpus.words(file['neg'][i])
    for w in movie_review:
      w = pre_process(w)
      if len(w) > 0:
        if w not in words_dic:
          words_dic[w] = 1
        else:
          words_dic[w] = words_dic[w] + 1
      else:
        continue
    i = i + 1
  words_dic.pop('') # remove space
  sorted_words_dic = sorted(words_dic.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)
  # add the top words to the list to be returned
  i = 0
  while i < top:
    iw.append(sorted_words_dic[i])
    i = i + 1
  return iw

# the input parameter is a list consisting of words
# and return a dictionary which includes useful words after pre-processing
def feature_selection(words):
  wd = {}
  for word in words:
    word = pre_process(word)
    # if len(word) > 0 and word in iw and word not in wd:
    if len(word) > 0 and word not in wd:
      wd[word] = True
  return wd

# collect several/some/many dict-type positive or negative movie reviews into a list
# for training or testing
# the returned list is composed of tuple
# every tuple is the ict-type positive or negative movie reviews + clf
# corpus, should be the 'mr'
# files, should be the 'new_mr'
# clf, is the classification: pos or neg
def collect_features(corpus, files, startID, endID, clf):
  col = []
  i = startID
  while i < endID:
    dic = feature_selection(corpus.words(files[clf][i]))
    tup = (dic, clf)
    col.append(tup)
    i = i + 1
  return col

#top = 3000
#iw = importantWords(mr, new_mr, top)
#print('selection the 3000 most important words')


train_start_id = 100
train_end_id = 900
test_start_id = 900
test_end_id = 1000

pos_train = collect_features(mr, new_mr, train_start_id, train_end_id, 'pos')
pos_test = collect_features(mr, new_mr, test_start_id, test_end_id, 'pos')

neg_train = collect_features(mr, new_mr, train_start_id, train_end_id, 'neg')
neg_test = collect_features(mr, new_mr, test_start_id, test_end_id, 'neg')

# whole sets
train_set = pos_train + neg_train
test_set = pos_test + neg_test

print('pre-processing and divide the movie reviews into training and testing set')
print('training set: 90% \ntesting set: 10%')
"""
print(len(neg_training))
print(len(neg_test))
print(len(pos_training))
print(len(pos_test))
print(len(train_set))
print(len(test_set))
"""
print()

step 2: 
there are 141 stopwords loaded
pre-processing and divide the movie reviews into training and testing set
training set: 90% 
testing set: 10%



In [ ]:
print('step 3:')
# text classification
# Use a Naive Bayes Classifier
from nltk.classify import NaiveBayesClassifier
print('text classification: Naive Bayes Text classification')

# Train a classifier on our training data.
classifier = NaiveBayesClassifier.train(train_set)
print('using NB classifier to train the model on 90% of movie reviews')

# get a certain moview review
# corpus, 'rm'
# files, 'new_rm'
# clf, 'pos' or 'neg'
# fileID, n-th
# startID, endID, the sub-string of the retrieved moview review
def get_mr_text(corpus, files, clf, fileID, startID=0, endID=None):
  words = list(corpus.words(files[clf][fileID]))
  return ' '.join(words[startID:endID])

#print(get_mr_text(mr, new_mr, 'pos', 900) )
#print(get_mr_text(mr, new_mr, 'neg', 900) )
print()

step 3:
text classification: Naive Bayes Text classification
using NB classifier to train the model on 90% of movie reviews



In [ ]:
print('step 4:')
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
print('evaluate trained model on 10% of movie reviews')

# pairs, after using the model to test, it will generate every testing case as a tuple
# such as: ( 'pos', 'pos' ), the first one is predicted one, and the latter one is actual one
# pos_label, 'pos' or 'neg'
# printout, True means to print it out and False means not
def evaluate(pairs, pos_label, printout):
    predicted, actual = zip(*pairs)
    precision = precision_score(actual,predicted,pos_label=pos_label)
    recall = recall_score(actual,predicted,pos_label=pos_label)
    accuracy = accuracy_score(actual,predicted)
    f1 = f1_score(actual,predicted,pos_label=pos_label)
    if printout:
      print_evaluation(precision, recall, accuracy, f1, pos_label)
    return (precision, recall, accuracy, f1)

def print_evaluation(precision, recall, accuracy, f1, pos_label):
    title =  'Evaluation with pos label = %s' % pos_label
    print(title)
    print('=' * len(title))
    print('{0:10s} {1:.2f} %'.format('Precision',precision*100))
    print('{0:10s} {1:.2f} %'.format('Recall',recall*100))
    print('{0:10s} {1:.2f} %'.format('Accuracy',accuracy*100))
    print('{0:10s} {1:.2f} %'.format('F1 score',f1*100))

pairs = [(classifier.classify(predicted), actual) for (predicted, actual) in test_set]
printout = True
precision, recall, accuracy, f1 = evaluate(pairs, 'pos', printout)
print()
precision, recall, accuracy, f1 = evaluate(pairs, 'neg', printout)
print()


step 4:
evaluate trained model on 10% of movie reviews
Evaluation with pos label = pos
Precision  63.40 %
Recall     97.00 %
Accuracy   70.50 %
F1 score   76.68 %

Evaluation with pos label = neg
Precision  93.62 %
Recall     44.00 %
Accuracy   70.50 %
F1 score   59.86 %



In [ ]:
print('step 5:')
print('testing the exclusive movie reviews with this model')
# new testing examples excluded from the move_reviews set
movie_review = 'Inception is the best movie ever'
prediction = classifier.classify(feature_selection(movie_review.split()))
print(prediction)
movie_review = "I don't know how anyone could sit through Inception"
prediction = classifier.classify(feature_selection(movie_review.split()))
print(prediction)
classifier.show_most_informative_features()

step 5:
testing the exclusive movie reviews with this model
neg
neg
Most Informative Features
               ludicrous = True              neg : pos    =     21.0 : 1.0
                 idiotic = True              neg : pos    =     16.3 : 1.0
             outstanding = True              pos : neg    =     15.0 : 1.0
                 chuckle = True              neg : pos    =     14.3 : 1.0
                  avoids = True              pos : neg    =     11.0 : 1.0
              astounding = True              pos : neg    =     11.0 : 1.0
                poignant = True              pos : neg    =     10.6 : 1.0
                 offbeat = True              pos : neg    =     10.3 : 1.0
             fascination = True              pos : neg    =     10.3 : 1.0
                religion = True              pos : neg    =      9.8 : 1.0


In [ ]:
def unigram_features (words):
    """
    This is the simplest possible feature representation of a document.

    Each word is a feature.
    """
    return dict((word, True) for word in words)

# the input parameter is a list consisting of words
# and return a dictionary which includes useful words after pre-processing
def feature_selection1(words):
  wd = {}
  for word in words:
    word = pre_process(word)
    # if len(word) > 0 and word in iw and word not in wd:
    if len(word) > 0:
      wd[word] = True
  return wd

def focusWords(word):
  adj_list = ['ing', 'ous', 'ic', 'ed', 'ive', 'ble', 'al', 'ant', 'ary', 'ful', 'less', 'y']
  for w in adj_list:
      if word.endswith(w):
          return True
  neg_list = ['no', 'nor', 'not', "too", 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'haven', 'isn', 'mightn', 'hasn', 'mustn', 'needn', 'shan', 'wasn', 'weren', 'wouldn']
  for w in neg_list:
      if word == w:
          return True
  if word.endswith("n't"):
      return True
  return False
def feature_selection_update(words):
  wd = {}
  for word in words:
    word = pre_process(word)
    if focusWords(word):
      wd[word] = True
  return wd

tmp_list = [ 'I', 'Love', 'you', 'I', 'Love', 'you', 'this', 'is', 'not', 'exciting', 'wouldnt',"wouldn't" ]
print(tmp_list)
print(unigram_features(tmp_list))
print(feature_selection1(tmp_list))
print(feature_selection_update(tmp_list))

['I', 'Love', 'you', 'I', 'Love', 'you', 'this', 'is', 'not', 'exciting', 'wouldnt', "wouldn't"]
{'I': True, 'Love': True, 'you': True, 'this': True, 'is': True, 'not': True, 'exciting': True, 'wouldnt': True, "wouldn't": True}
{'i': True, 'love': True, 'not': True, 'exciting': True, 'wouldnt': True, "wouldn't": True}
{'not': True, 'exciting': True, "wouldn't": True}
